In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = np.load('/kaggle/input/datasetchallengeann2/training_dataset/training_data.npy')
data.shape

In [ ]:
categories = np.load('/kaggle/input/datasetchallengeann2/training_dataset/categories.npy')
valid_indices = np.load('/kaggle/input/datasetchallengeann2/training_dataset/valid_periods.npy')
print(categories.shape)
print(valid_indices.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_random_timeseries(data, labels, types=('A', 'B', 'C', 'D', 'E')):
    """
    Plot one random time series for each type specified in 'types'.

    Parameters:
    - data: numpy array of shape (num_time_series, time_series_length)
    - labels: numpy array of shape (num_time_series,) representing the type for each time series
    - types: tuple specifying the types to plot (default is ('A', 'B', 'C', 'D', 'E'))
    """
    # Create a figure with subplots for each type
    fig, axs = plt.subplots(len(types), 1, figsize=(10, 2 * len(types)), sharex=True, sharey=True)

    # Loop through each type and plot a random time series of that type
    for i, t in enumerate(types):
        # Get indices of time series with the specified type
        type_indices = np.where(labels == t)[0]
        
        # Select a random index for the given type
        random_index = np.random.choice(type_indices)
        
        # Extract the time series data
        time_series = data[random_index, valid_indices[random_index, 0]:]
        
        # Plot the time series
        axs[i].plot(time_series)
        axs[i].set_title(f'Type {t}')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

plot_random_timeseries(data, categories)

<h3>Training on model F</h3>

In [ ]:
type_F_mask = (categories == "F") 
data_type_F= data[type_F_mask]
valid_indices_type_F = valid_indices[type_F_mask]

In [ ]:
data_type_F.shape

In [ ]:

# Randomly select 10 time series indices
random_indices = np.random.choice(data_type_F.shape[0], 10, replace=False)

# Plot each selected time series in a separate subplot
num_plots = len(random_indices)
fig, axs = plt.subplots(num_plots, 1, figsize=(10, 2 * num_plots))

for i, index in enumerate(random_indices):
    axs[i].plot(data_type_F[index, valid_indices_type_F[index, 0]:])
    axs[i].set_title(f'Time Series {index}')
    axs[i].set_xlabel('Time')
    axs[i].set_ylabel('Value')

plt.tight_layout()
plt.show()

In [ ]:
def build_sequences(df, window=100, stride=20, telescope=100):
    assert window % stride == 0
    dataset = []
    labels = []
    df = np.expand_dims(df, axis = -1)
    temp_df = np.copy(df)
    padding_check = len(df)%window

    if(padding_check != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros(( padding_len, temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_df[idx+window:idx+window+telescope])
    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
from sklearn.model_selection import train_test_split
X_train_raw, X_validation_raw, valid_indices_train, valid_indices_validation = train_test_split(data_type_F, valid_indices_type_F, test_size = 0.2)
print(X_train_raw.shape, X_validation_raw.shape)

In [ ]:
print(valid_indices_train.shape)

In [ ]:
lengths = []
for i in range(len(valid_indices_type_F)):
    lengths.append(valid_indices_type_F[i, 1]-valid_indices_type_F[i, 0])
lengths = np.array(lengths)

In [ ]:
print(min(lengths), max(lengths))

In [ ]:
print(np.count_nonzero(lengths<42))

In [ ]:
autoregressive_telescope=3

In [ ]:
X_train = []
y_train = []
for i, time_series in enumerate(X_train_raw):
    dataset_tmp, labels_tmp = build_sequences(time_series[valid_indices_train[i, 0]:], window = 24, stride = 3, telescope = autoregressive_telescope)
    X_train.extend(dataset_tmp)
    y_train.extend(labels_tmp)
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_val = []
y_val = []
for i, time_series in enumerate(X_validation_raw):
    dataset_tmp, labels_tmp = build_sequences(time_series[valid_indices_validation[i, 0]:], window = 24, stride = 3, telescope = autoregressive_telescope)
    X_val.extend(dataset_tmp)
    y_val.extend(labels_tmp)
X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
def inspect_multivariate(X, y, telescope, idx=None):
    random_indices = np.random.choice(X.shape[0], 10, replace=False)

    # Plot each selected time series in a separate subplot
    num_plots = len(random_indices)
    fig, axs = plt.subplots(num_plots, 1, figsize=(10, 2 * num_plots))

    for i, index in enumerate(random_indices):
        axs[i].plot(np.arange(len(X[0,:])), X[index,:])
        axs[i].scatter(np.arange(len(X[0,:]), len(X[0,:])+telescope), y[index,:], color='orange')
        axs[i].set_title(f'Time Series {index}')
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Value')

    plt.tight_layout()
    plt.show()

In [ ]:
inspect_multivariate(X_train, y_train, autoregressive_telescope)

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):#merge convolution and lstm, to extract features and correlate in the time domain
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Add a Bidirectional LSTM layer with 64 units
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)

    # Add a 1D Convolution layer with 128 filters and a kernel size of 3
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv')(x)

    # Add a final Convolution layer to match the desired output shape
    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)

    # Calculate the size to crop from the output to match the output shape
    crop_size = output_layer.shape[1] - output_shape[0]

    # Crop the output to the desired length
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)#CROPPO AL POSTO DI USARE AD ESEMPIO MAX POOLING IN QUANTO AVERE DIMENSIONI STRANE SAREBBE PROBLEMATICO CON MAX POOLING

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
model.save('modeltypeF.h5')